# Minimal YAML - Clean, Readable Schema Serialization

The `minimal_yaml` function provides opinionated YAML serialization optimized for readability and simplicity. It's designed for configuration files, schema documentation, and human-readable data interchange.

**Core Features:**
- **Clean Output**: No anchors/aliases (`&id001`, `*id001`) - pure, readable YAML
- **Smart Strings**: Multiline text uses block scalars (`|`), single-line uses plain style
- **Auto-Pruning**: Removes empty containers (None, empty strings/lists/dicts) by default
- **JSON Auto-Parse**: Accepts JSON strings and converts them to YAML
- **Configurable**: Control indentation, line width, key sorting
- **Type-Safe**: Preserves 0 and False (not considered empty)

In [1]:
from lionherd_core.libs.schema_handlers import minimal_yaml

## 1. Basic Usage

Convert Python objects to clean YAML strings.

In [2]:
# Simple dictionary
data = {
    "name": "Alice",
    "age": 30,
    "active": True,
}

yaml_str = minimal_yaml(data)
print(yaml_str)

name: Alice
age: 30
active: true



In [3]:
# Nested structure
config = {
    "database": {
        "host": "localhost",
        "port": 5432,
        "credentials": {
            "username": "admin",
            "password": "secret",
        },
    },
    "features": ["auth", "logging", "metrics"],
}

print(minimal_yaml(config))

database:
  host: localhost
  port: 5432
  credentials:
    username: admin
    password: secret
features:
- auth
- logging
- metrics



## 2. Multiline String Handling

Multiline strings automatically use block scalar style (`|`) for readability.

In [4]:
# Single-line string - plain style
single_line = {"message": "Hello, World!"}
print("Single-line:")
print(minimal_yaml(single_line))

# Multiline string - block scalar style
multiline = {
    "description": """This is a long description
that spans multiple lines.

It includes paragraphs and formatting."""
}
print("Multiline:")
print(minimal_yaml(multiline))

Single-line:
message: Hello, World!

Multiline:
description: |-
  This is a long description
  that spans multiple lines.

  It includes paragraphs and formatting.



## 3. Empty Value Pruning

By default, `drop_empties=True` removes None, empty strings, empty containers. Preserves 0 and False.

In [5]:
# Data with empty values
messy_data = {
    "name": "Bob",
    "email": "",  # Empty string - will be removed
    "age": 0,  # Zero - will be kept
    "active": False,  # False - will be kept
    "tags": [],  # Empty list - will be removed
    "metadata": None,  # None - will be removed
    "config": {"key": None},  # Nested empty - will be removed
}

print("With pruning (drop_empties=True):")
print(minimal_yaml(messy_data))

print("Without pruning (drop_empties=False):")
print(minimal_yaml(messy_data, drop_empties=False))

With pruning (drop_empties=True):
name: Bob
age: 0
active: false

Without pruning (drop_empties=False):
name: Bob
email: ''
age: 0
active: false
tags: []
metadata: null
config:
  key: null



## 4. JSON Auto-Parsing

Pass JSON strings directly - they're automatically parsed and converted to YAML.

In [6]:
# JSON string input
json_str = '{"user": "alice", "roles": ["admin", "editor"], "level": 5}'

print("Input JSON string:")
print(json_str)

print("\nConverted to YAML:")
print(minimal_yaml(json_str))

Input JSON string:
{"user": "alice", "roles": ["admin", "editor"], "level": 5}

Converted to YAML:
user: alice
roles:
- admin
- editor
level: 5



In [7]:
# Invalid JSON is treated as plain string
not_json = "This is not valid JSON"

print("Non-JSON input (treated as string):")
print(minimal_yaml(not_json))

Non-JSON input (treated as string):
This is not valid JSON
...



## 5. Configuration Options

Customize indentation, line width, and key sorting.

In [8]:
data = {
    "zebra": "last",
    "apple": "first",
    "middle": "value",
    "nested": {"z": 1, "a": 2},
}

# Default indentation (2 spaces)
print("Default indent (2):")
print(minimal_yaml(data))

# Custom indentation (4 spaces)
print("Custom indent (4):")
print(minimal_yaml(data, indent=4))

Default indent (2):
zebra: last
apple: first
middle: value
nested:
  z: 1
  a: 2

Custom indent (4):
zebra: last
apple: first
middle: value
nested:
    z: 1
    a: 2



In [9]:
# Sorted keys (default: False)
print("Unsorted (default):")
print(minimal_yaml(data))

print("Sorted alphabetically:")
print(minimal_yaml(data, sort_keys=True))

Unsorted (default):
zebra: last
apple: first
middle: value
nested:
  z: 1
  a: 2

Sorted alphabetically:
apple: first
middle: value
nested:
  a: 2
  z: 1
zebra: last



## 6. No Anchors/Aliases

Standard YAML uses anchors (`&id`) and aliases (`*id`) for repeated objects. MinimalDumper disables this for cleaner output.

In [10]:
import yaml

# Shared object
shared = {"x": 1, "y": 2}
data_with_refs = {
    "first": shared,
    "second": shared,  # Same object referenced twice
}

# Standard YAML (with anchors/aliases)
print("Standard YAML (with anchors/aliases):")
print(yaml.dump(data_with_refs, default_flow_style=False))

# minimal_yaml (clean, no anchors)
print("minimal_yaml (no anchors):")
print(minimal_yaml(data_with_refs))

Standard YAML (with anchors/aliases):
first: &id001
  x: 1
  y: 2
second: *id001

minimal_yaml (no anchors):
first:
  x: 1
  y: 2
second:
  x: 1
  y: 2



## 7. Complex Nested Structures

Handle deeply nested data with mixed types.

In [11]:
complex_schema = {
    "api_version": "v1",
    "metadata": {
        "name": "user-service",
        "labels": {"env": "prod", "team": "platform"},
    },
    "spec": {
        "replicas": 3,
        "template": {
            "containers": [
                {
                    "name": "app",
                    "image": "myapp:1.0",
                    "env": [
                        {"name": "DB_HOST", "value": "localhost"},
                        {"name": "DB_PORT", "value": 5432},
                    ],
                }
            ]
        },
    },
}

print(minimal_yaml(complex_schema))

api_version: v1
metadata:
  name: user-service
  labels:
    env: prod
    team: platform
spec:
  replicas: 3
  template:
    containers:
    - name: app
      image: myapp:1.0
      env:
      - name: DB_HOST
        value: localhost
      - name: DB_PORT
        value: 5432



## 8. Edge Cases and Type Preservation

Verify behavior with edge cases: empty containers, zero values, boolean False.

In [12]:
edge_cases = {
    "zero_int": 0,
    "zero_float": 0.0,
    "false_bool": False,
    "empty_string": "",
    "whitespace": "   ",
    "none_value": None,
    "empty_list": [],
    "empty_dict": {},
    "empty_tuple": (),
    "list_with_empties": [1, None, "", 2, []],
    "dict_with_empties": {"keep": 1, "remove1": None, "remove2": ""},
}

print("Edge cases (with pruning):")
print(minimal_yaml(edge_cases))

print("\nEdge cases (without pruning):")
print(minimal_yaml(edge_cases, drop_empties=False))

Edge cases (with pruning):
zero_int: 0
zero_float: 0.0
false_bool: false
list_with_empties:
- 1
- 2
dict_with_empties:
  keep: 1


Edge cases (without pruning):
zero_int: 0
zero_float: 0.0
false_bool: false
empty_string: ''
whitespace: '   '
none_value: null
empty_list: []
empty_dict: {}
empty_tuple: []
list_with_empties:
- 1
- null
- ''
- 2
- []
dict_with_empties:
  keep: 1
  remove1: null
  remove2: ''



## 9. Practical Use Cases

Real-world scenarios where minimal_yaml shines.

In [13]:
# Use case 1: Configuration file generation
app_config = {
    "server": {
        "host": "0.0.0.0",
        "port": 8000,
        "workers": 4,
    },
    "logging": {
        "level": "INFO",
        "format": "json",
        "handlers": ["console", "file"],
    },
    "database": {
        "url": "postgresql://localhost/mydb",
        "pool_size": 10,
        "timeout": 30,
    },
}

print("Application Config:")
print(minimal_yaml(app_config))

Application Config:
server:
  host: 0.0.0.0
  port: 8000
  workers: 4
logging:
  level: INFO
  format: json
  handlers:
  - console
  - file
database:
  url: postgresql://localhost/mydb
  pool_size: 10
  timeout: 30



In [14]:
# Use case 2: API schema documentation
api_schema = {
    "openapi": "3.0.0",
    "info": {"title": "User API", "version": "1.0.0"},
    "paths": {
        "/users": {
            "get": {
                "summary": "List users",
                "responses": {
                    "200": {
                        "description": "Successful response",
                        "content": {
                            "application/json": {
                                "schema": {"type": "array", "items": {"type": "object"}}
                            }
                        },
                    }
                },
            }
        }
    },
}

print("OpenAPI Schema:")
print(minimal_yaml(api_schema))

OpenAPI Schema:
openapi: 3.0.0
info:
  title: User API
  version: 1.0.0
paths:
  /users:
    get:
      summary: List users
      responses:
        '200':
          description: Successful response
          content:
            application/json:
              schema:
                type: array
                items:
                  type: object



In [15]:
# Use case 3: Clean data export with pruning
user_data = [
    {"name": "Alice", "email": "alice@example.com", "phone": "", "tags": []},
    {"name": "Bob", "email": "", "phone": "555-1234", "tags": None},
    {"name": "Charlie", "email": "charlie@example.com", "phone": "555-5678", "tags": ["vip"]},
]

print("User Data Export (cleaned):")
print(minimal_yaml(user_data))

User Data Export (cleaned):
- name: Alice
  email: alice@example.com
- name: Bob
  phone: 555-1234
- name: Charlie
  email: charlie@example.com
  phone: 555-5678
  tags:
  - vip



## Summary Checklist

**minimal_yaml Essentials:**
- ✅ Clean output with no anchors/aliases (`&id`, `*id`)
- ✅ Multiline strings use block scalar (`|`) style
- ✅ Auto-prunes empty values (None, `""`, `[]`, `{}`)
- ✅ Preserves meaningful zeros and False
- ✅ Auto-parses JSON strings to YAML
- ✅ Configurable indentation and key sorting
- ✅ Infinite line width by default (no line wrapping)
- ✅ Graceful fallback for invalid JSON input

**When to Use:**
- Configuration file generation
- Schema documentation
- Human-readable data export
- Clean API responses in YAML format

**Key Parameters:**
- `drop_empties`: Remove empty values (default: True)
- `indent`: Spaces per level (default: 2)
- `line_width`: Max line length (default: unlimited)
- `sort_keys`: Alphabetical key sorting (default: False)

**Next Steps:**
- Pair with `Element.to_dict()` for serializing lionherd entities
- Use in config management workflows
- Integrate with schema validation pipelines